In [79]:
import numpy as np
from scipy import signal as sg
import scipy.misc
import matplotlib.pyplot as plt
%matplotlib inline

In [85]:
def hog(image, bins, pixels_per_cell, cells_per_block, small_constant = 0.000000001):
    # convolution Matrix
    kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])   

    """Stage One - Computing first order image gradients + magnitude and orientation"""    
    #Extract Histogram of Oriented Gradients (HOG) for a given image.
    # array of horizontal gradients for each pixel
    grad_x = sg.convolve2d(image, kernel)

    # array of vertical gradients for each pixel
    grad_y = sg.convolve2d(image, kernel.T)

    # combining x and y gradients by calculating the magnitude
    # resulting array contains the gradient magnitude for each pixel
    grad_mag_raw = np.sqrt(np.power(grad_x, 2)+np.power(grad_y, 2))
    grad_mag  = grad_mag_raw/np.max(grad_mag_raw)

    # Finding the orientation of the gradeint for each pixel    
    grad_ang_rad = np.arctan2(grad_y, grad_x)

    # Converting angles from -pi to pi to 0 to 360 degrees
    grad_ang_deg = 180*(grad_ang_rad + np.pi)/np.pi   
    
    """Stage Two - Cell Orientation Binning """
    
    # Defining values
    s_row, s_col = image.shape[:2]     #y and x lengths of images
    c_row, c_col = pixels_per_cell     #y and x lengths of cell
    b_row, b_col = cells_per_block     #y and x lengths of block

    n_cells_row = int(s_row / c_row)  # number of cells along row-axis
    n_cells_col = int(s_col / c_col)  # number of cells along col-axis

    w = (360/bins)  # Cell width    
    cells = []      # empty list to append cells   

    # Fill histogram via bilinear interpolation
    # creating histograms for each cell
    for r in range(c_row, s_row+1, c_row):
        for c in range(c_col, s_col+1, c_col):
            histogram = np.zeros(bins)              # Initialize histogram
            cell = grad_ang_deg[r - c_row:r, c - c_col:c]   # cell array of pixel orientations
            cell_mags = grad_mag[r - c_row:r, c - c_col:c].flatten()  # cell array of pixel magnitudes

            # For each pixel in the cell
            for i, x in enumerate(cell.flatten()):
                
                # Select Histogram Bins
                j0 = int(np.floor((x/w)-0.5)%bins)
                j1 = int((j0+1)%bins)
                
                # Special cases for when two bins are 7 and 0
                if (j0 == 7) and (x <= 22.5):
                    vj0 = cell_mags[i]*((w*(j1+0.5)-x)/w)
                    vj1 = cell_mags[i]*((x+360-w*(j0+0.5))/w)
                 
                elif (j0 == 7) and (x >= 337.5):
                    vj0 = cell_mags[i]*((w*(j1+0.5)-x+360)/w)
                    vj1 = cell_mags[i]*((x-w*(j0+0.5))/w)
                
                # All other cases
                else:    
                    vj0 = cell_mags[i]*((w*(j1+0.5)-x)/w)
                    vj1 = cell_mags[i]*((x-w*(j0+0.5))/w)   

                # Adding votes to histogram cell 
                histogram[j0] += vj0
                histogram[j1] += vj1
                
            # Adding cell (list of histograms) to the cell list    
            cells.append(histogram)
        
    """Stage Three - Creating Blocks """
    cells_ordered = np.reshape(cells, (n_cells_row, n_cells_col, 8))/2  #unflattening list and diving by two since other implementations are between 0 and 180 not 0 and 360

    blocks = []
    for r in range(b_row, n_cells_row):
        for c in range(b_col, n_cells_row):
            block = cells_ordered[r - b_row:r, c - b_col:c, :]
            block_f = block.flatten()
            block_n = (block_f)/np.sqrt(np.power(np.around(block_f), 2) + small_constant)
            blocks.append(block_n)
    
    h_normed = (h)/np.sqrt(np.power(np.around(h), 2) + small_constant)
    
    return h_normed

    


In [86]:
image = hog(scipy.misc.face(True)[:, 256:], 8, pixels_per_cell = (24, 24), cells_per_block =(2, 2))

In [87]:
print (image)

[ 1.05763256  0.9857208   0.99297966 ...,  1.38866764  1.09641113
  0.99998733]
